# Stealth address
Ce notebook explicite le notebook [original](https://github.com/nerolation/EIP-Stealth-Address-ERC) mais avec une unique clé de dépense.

### Rappel sur la cryptographie sur courbe elliptique 
Une courbe elliptique est définie par une équation de type: 


$y^2= ax^3 + bx + c$

avec a,b,c fixes, x et y les variables du corps fini.
Il s'agit donc d'une fonction à deux variables: chaque point est défini par ses coordonnées x et y : (x,y).

Pour définir un système cryptographique basé sur les courbes elliptiques, on choisit une courbe elliptique en fixant a b et c, on choisit un point de la courbe G, qu'on nomme le générateur et qui est publique, et on choisit un maximum du corps fini (pour les modulos). 

On peut additionner deux points facilement mais la multiplication est particulière car multiplier G par 5 par exemple revient à additionner G 5 fois.

Dans les systèmes cryptographiques, on utilise des fonctions trapdoors, c'est à dire qu'il est facile de réaliser une opération dans un sens mais quasiment impossible dans l'autre sens. 
En l'occurence, il est facile de multiplier G par 5 pour obtenir un point P, mais à partir de P il n'est pas possible de récupérer le 5 initial. C'est le problème du logarithme discret. 

Pour une introduction plus détaillée: [lien](https://andrea.corbellini.name/2015/05/17/elliptic-curve-cryptography-a-gentle-introduction/)

Cela nous permet de définir une clé privée s, et de communiquer la clé publique S=G*s . A partir de S, il n'est pas possible de récupérer s. 

In [21]:
#!python -m pip install py_ecc pysha3 eth_account

In [2]:
import hashlib
from py_ecc.secp256k1 import *
import sha3
from eth_account import Account

## Expéditeur


Ici l'expéditeur est Alice, elle veut envoyer des tokens à Bob. Pour cela, elle commence par générer une clé privée éphémère $s_{eph}$ et en déduit sa clé publique éphèmère $S_{eph}$. Cette clé ne servira qu'une fois, pour ce transfert. Elle garde secrète sa clé privée éphémère et publie, envoie au registre global des clés éphémères, la clé publique éphémère $S_{eph}$.

$S_{eph} = G*s_{eph}$

In [3]:
# privkey: 0xd952fe0740d9d14011fc8ead3ab7de3c739d3aa93ce9254c10b0134d80d26a30
# address: 0x3CB39EA2f14B16B69B451719A7BEd55e0aFEcE8F
a_eph = int(0xd952fe0740d9d14011fc8ead3ab7de3c739d3aa93ce9254c10b0134d80d26a30) # private key
A_eph = secp256k1.privtopub(a_eph.to_bytes(32, "big")) # public key
A_eph

(22246744184454969143801186698733154500632648736073949898323976612504587645286,
 110772761940586493986212935445517909380300793379795289150161960681985511655321)

## Destinataire

Ici le receveur, Bob, génère une clé privée, et en déduit sa clé publique. On a donc 2 clés : 

*   La clé privée de dépense b_spend
*   La clé publique de dépense B_spend

Dans le blog de Vitalik, cette clé publique hashée correspond à la stealth meta-address.


$B_{spend} = G*b_{spend}$

In [4]:
# privkey: 0x0000000000000000000000000000000000000000000000000000000000000001
# address: 0x7E5F4552091A69125d5DfCb7b8C2659029395Bdf
b_spend = int(0x0000000000000000000000000000000000000000000000000000000000000003) # private key

B_spend = secp256k1.privtopub(b_spend.to_bytes(32, "big")) # public key
B_spend

(112711660439710606056748659173929673102114977341539408544630613555209775888121,
 25583027980570883691656905877401976406448868254816295069919888960541586679410)

## Principe 
Maintenant que Alice a la clé éphémère et Bob a sa clé privée de dépense, le but est de construire une adresse où Alice peut envoyer des tokens que seul Bob peut récupérer, sans dévoiler évidemment son adresse principale. 
Pour cela, il faut: 
1.   Se mettre d'accord sur une addresse, clé publique, éphémère sur laquelle Alice et Bob vont communiquer. (à la Diffie Hellman)
2.   faire en sorte que seul Bob connaisse la clé privée de cette clé publique éphémère. 

Pour rappel, Diffie Hellman permet à Alice et Bob de se mettre d'accord sur une clé que seuls eux-même connaissent, de manière sécurisée. Cela peut servir ensuite pour passer en cryptographie symétrique par exemple. La méthode des adresses stealth en est inspirée mais la modifie pour que seul Bob connaisse la clé privée de l'adresse convenue.  

## L'expéditeur Alice calcule l'adresse stealth de Bob 

Si $B_{spend}$ est la clé publique de Bob et $a_{eph}$ la clé privée éphémère d'Alice, Alice peut déterminer l'adresse partagée comme ceci:

$Shared_{key}=B_{spend} * a_{eph}$. 

Elle combine sa clé privée avec la clé publique de dépense de Bob.

Remarque: Cette adresse est partagée car Bob peut également retomber sur le même resultat d'une autre manière:
$Shared_{key}=b_{spend} * A_{eph}$. 
Il peut combiner sa clé privé de dépense avec la clé publique éphémère d'Alice qu'il a récupéré du registre. 
Cela s'explique par l'associativité de la multiplication:

$Shared_{key} \\ 
= G * b_{spend} * a_{eph} = (G*b_{spend})*a_{eph}= B_{spend} * a_{eph} \\
= G * b_{spend} * a_{eph} = b_{spend}*(G*a_{eph}) = b_{spend} * A_{eph}$

Alice et Bob se mettent donc d'accord sur une clé/adresse commune, ce qui correspond à du Diffie Hellman.




Pour cela, elle calcule d'abord le secret partagé: 


In [5]:
Shared = secp256k1.multiply(B_spend, a_eph)
Shared

(52736289061527850298314042255291833441398324762320546151491597132554285392820,
 48049573440383169888492387607745136363991885859721651081432779646157521013267)

In [6]:
assert Shared == secp256k1.multiply(A_eph, b_spend)

Alice hashe ensuite cette clé partagée

In [7]:
Shared_hex = sha3.keccak_256(Shared[0].to_bytes(32, "big") 
                        + Shared[1].to_bytes(32, "big")
                       ).hexdigest()
Shared_hashed = bytearray.fromhex(Shared_hex)


Maintenant, il faut modifier cette clé pour que seul Bob connaisse la clé privée. Pour cela Alice la hashe et la combine avec la clé publique de dépense de Bob. 

$ Stealth_{key}=B_{spend} + G * hash(Shared_{key})$


In [8]:
Stealth_key = secp256k1.add(B_spend, secp256k1.privtopub(Shared_hashed))
A_stealth_address = "0x"+ sha3.keccak_256(Stealth_key[0].to_bytes(32, "big")
                            +Stealth_key[1].to_bytes(32, "big")
                           ).hexdigest()[-40:]
A_stealth_address

'0x2a7fc499f8179c162256caf6946592b03c991fc4'

#### Alice envoie les tokens à cette adresse stealth  

#### Ensuite elle envoie la clé publique éphémère et l'adresse stealth au registre de clés éphémères.

In [9]:
A_eph, A_stealth_address

((22246744184454969143801186698733154500632648736073949898323976612504587645286,
  110772761940586493986212935445517909380300793379795289150161960681985511655321),
 '0x2a7fc499f8179c162256caf6946592b03c991fc4')

## Bob veut maintenant récupérer les fonds

Pour cela il calcule d'abord la clé partagée à l'aide de sa clé privée de dépense:

$Shared_{key}= G * b_{spend} * a_{eph} =b_{spend}*(G*a_{eph}) = b_{spend} * A_{eph}$

Comme Alice il hashe cette clé partagée et ajoute sa clé publique de dépense:

$ Stealth_{key}=B_{spend} + G * hash(Shared_{key})$


In [10]:
Shared_key = secp256k1.multiply(A_eph, b_spend)
Shared_hex = sha3.keccak_256(Shared_key[0].to_bytes(32, "big")+Shared_key[1].to_bytes(32, "big")).hexdigest()
Shared_hashed = bytearray.fromhex(Shared_hex)

B_stealth = secp256k1.add(B_spend, secp256k1.privtopub(Shared_hashed))
B_stealth_address  = "0x"+ sha3.keccak_256(Stealth_key[0].to_bytes(32, "big")
                                        + Stealth_key[1].to_bytes(32, "big")
                                        ).hexdigest()[-40:]
B_stealth_address

'0x2a7fc499f8179c162256caf6946592b03c991fc4'

Vérifions maintenant que l'adresse calculée par Alice est bien la même que celle calculée par Bob

In [11]:
B_stealth_address == A_stealth_address

True

## Calculer la clé privée de l'adresse stealth de Bob

De son côté, seul Bob peut récupérer les tokens déposés dans l'adresse stealth grâce à sa clé privée de dépense :
$ Stealth_{address}=B_{spend} + G * hash(Shared_{key})=b_{spend}*G+G * hash(Shared_{key})= G*(b_{spend}+hash(Shared_{key}))$

On voit qu'il faut multiplier G par $(b_{spend}+hash(Shared_{key}))$ pour obtenir l'adresse stealth, **c'est donc la clé privée correspondant à la clé publique stealth, connue uniquement de Bob qui connait $b_{spend}$**

$b_{stealth}=b_{spend}+hash(Shared_{key})$

In [12]:
Shared = secp256k1.multiply(A_eph, b_spend)
Shared_hex = sha3.keccak_256(Shared[0].to_bytes(32, "big")+Shared[1].to_bytes(32, "big")).hexdigest()
b_stealth = b_spend + int(Shared_hex, 16)
b_stealth

29716313209935704867209704493295016641057490968149592765059958401802925033007

Il calcule ensuite la clé publique 
$B_{stealth} = b_{stealth}*G$

In [13]:
# Recipient has private key to ...
B_stealth = secp256k1.privtopub(b_stealth.to_bytes(32, "big"))
B_stealth

(49528576054990901848060147798521484203360043839687519115038728561895435784037,
 105959932460373769028909069596244169130086221657223409932600679158517405683805)

Il met finalement cette clé de stealth sous forme d'adresse.

In [14]:
B_stealthAddress_d  = "0x"+ sha3.keccak_256(B_stealth[0].to_bytes(32, "big")
                                        + B_stealth[1].to_bytes(32, "big")
                                        ).hexdigest()[-40:]
B_stealthAddress_d

'0x2a7fc499f8179c162256caf6946592b03c991fc4'

In [15]:
Account.from_key((b_stealth).to_bytes(32, "big")).address

'0x2A7fC499f8179C162256CAF6946592B03c991fc4'

## Optionnel: Alice peut aider Bob à scanner le registre des clés en ajoutant le premier byte de $h(Shared_{key})$

In addition to S and stA, the sender also broadcasts the first byte of h(Q)

In [16]:
Shared_hashed[0]

65

The recipient can do the the same a before without one EC Multiplication, one EC Addition and on Public Key to Address Conversion in order to check being a potential recipient.

In [17]:
Shared_derived = secp256k1.multiply(A_eph, b_spend)
Shared_hex_derived = sha3.keccak_256(Shared_derived[0].to_bytes(32, "big")
                                +Shared_derived[1].to_bytes(32, "big")
                               ).hexdigest()
Shared_hashed_derived = bytearray.fromhex(Shared_hex_derived)

Check view tag

In [18]:
run = Shared_hashed[0] == Shared_hashed_derived[0] 
run

True

In [19]:
if run:
    B_stealth = secp256k1.add(B_spend, secp256k1.privtopub(Shared_hashed))
    B_stealthAddress  = "0x"+ sha3.keccak_256(Stealth_key[0].to_bytes(32, "big")
                                            + Stealth_key[1].to_bytes(32, "big")
                                            ).hexdigest()[-40:]
B_stealthAddress

'0x2a7fc499f8179c162256caf6946592b03c991fc4'

In [20]:
B_stealthAddress==A_stealth_address

True